In [1]:
from keras import backend as K
import tensorflow as tf
from math import ceil
import numpy as np
import math
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy32
from bounding_box_utils.bounding_box_utils import iou_float, convert_coordinates
import timeit

Using TensorFlow backend.


In [2]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
K.set_session(sess)
K.clear_session()

def eva(x):
    return(K.eval(x))

In [3]:
def tf_intersection_area_(boxes1, boxes2, coords='corners', mode='outer_product', border_pixels='half'):
    m = tf.shape(boxes1)[0]
    n = tf.shape(boxes2)[0]
    
    xmin = 0
    ymin = 1
    xmax = 2
    ymax = 3

    d = 0
    
    box1xminxmax = tf.slice(boxes1, [0, 0], [m, xmax])
    box2xminxmax = tf.slice(boxes2, [0, 0], [n, xmax])                      
                            
    min_xy = tf.maximum(tf.tile(tf.expand_dims(box1xminxmax,1),[1, n, 1]),
                        tf.tile(tf.expand_dims(box2xminxmax,0),[m, 1, 1]))
    
    box1xmaxymax = tf.slice(boxes1, [0, 2], [m, 2])
    box2xmaxymax = tf.slice(boxes2, [0, 2], [n, 2])                      

    max_xy = tf.minimum(tf.tile(tf.expand_dims(box1xmaxymax,1),[1, n, 1]),
                    tf.tile(tf.expand_dims(box2xmaxymax,0),[m, 1, 1]))

    side_lengths = tf.maximum(tf.cast(0, dtype=tf.int64), tf.add(tf.subtract(max_xy,min_xy),d))

    return tf.multiply(side_lengths[:,:,0],side_lengths[:,:,1])

In [4]:
def tf_iou(boxes1, boxes2, coords='centroids', mode='outer_product', border_pixels='half'):

    intersection_areas = tf_intersection_area_(boxes1, boxes2, coords=coords, mode=mode)
    m = tf.shape(boxes1)[0]
    n = tf.shape(boxes2)[0]
    
    xmin = 0
    ymin = 1
    xmax = 2
    ymax = 3

    d = 0
    
    boxes1_areas = tf.tile(tf.expand_dims(tf.multiply(tf.add(tf.subtract(boxes1[:,xmax],boxes1[:,xmin]),d),tf.add(tf.subtract(boxes1[:,ymax],boxes1[:,ymin]),d)), 1), [1,n])
    boxes2_areas = tf.tile(tf.expand_dims(tf.multiply(tf.add(tf.subtract(boxes2[:,xmax],boxes2[:,xmin]),d),tf.add(tf.subtract(boxes2[:,ymax],boxes2[:,ymin]),d)), 0), [m,1])
    
    union_areas = tf.subtract(tf.add(boxes1_areas,boxes2_areas),intersection_areas)
    op = tf.divide(intersection_areas,union_areas)
    return op

In [5]:
y = np.load('y_pred.npy')
x = np.load('y_true.npy')

y = tf.convert_to_tensor(y)
x = tf.convert_to_tensor(x)

t = tf_iou(x,y)

In [46]:
def match_bipartite_greedy_tf(weight_matrix):
    weight_matrix = tf.identity(weight_matrix)
    num_ground_truth_boxes = tf.cast(tf.shape(weight_matrix)[0], dtype=tf.int32)
    all_gt_indices = tf.range(num_ground_truth_boxes)
    matches = tf.zeros([num_ground_truth_boxes], tf.int32) 
    

    anchor_indices = tf.argmax(weight_matrix, axis=1)
    anchor_indices = tf.cast(anchor_indices,dtype=tf.int32)
    print("weight_matrix: ", K.eval(weight_matrix))
    print("all_gt_indices: ", K.eval(all_gt_indices))
    print("anchor_indices: ", K.eval(anchor_indices))
    
    print("weight_matrix: ", K.eval(weight_matrix).shape)
    overlaps = tf.gather(weight_matrix,[all_gt_indices, anchor_indices])
    
    print("overlaps: ", K.eval(overlaps).shape)

    ground_truth_index = tf.argmax(overlaps)
    anchor_index = anchor_indices[ground_truth_index]
    matches[ground_truth_index] = anchor_index
    weight_matrix[ground_truth_index] = 0
    weight_matrix[:,anchor_index] = 0
    return matches
    

In [47]:
match_bipartite_greedy_tf(t)

weight_matrix:  [[-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 ...
 [-0. -0. -0. ... -0. -0. -0.]
 [ 0. -0.  0. ... -0. -0.  0.]
 [-0. -0. -0. ...  0.  0.  0.]]
all_gt_indices:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
anchor_indices:  [ 0  0  0 16  0  0  0  0  0  0  0 35  0  0  0  0  0  0  0 33  0  0  0  0
  0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 16 26
 26 35]
weight_matrix:  (50, 50)
overlaps:  (2, 50, 50)


ValueError: Shape must be rank 1 but is rank 3 for 'strided_slice_42' (op: 'StridedSlice') with input shapes: [50], [1,50,50], [1,50,50], [1].